<h2>Market Study - Chicken Exports</h2>

<a href="https://openclassrooms.com/en/" >Openclassrooms</a>, Data Analyst Course
<br>Project 5 - Michael Orange


<hr>

A company in the food industry specialized in chicken is looking at expanding worldwide. All countries are considered. 
<br>The main objective is to provide the company with data ensuring it will collaborate with the most promising export market countries. 

Grouping countries in 'clusters' based on their similarity is required. 
<br>Default variables are: 
- population growth, 
- calory supply per capita and protein supply per capita, 
- prevalence of animal proteins in the mix. 

Additional relevant variable might be included (ex. GDP per capita)

<hr>

**Section 1** [Importing FAOSTAT Datasets](#import)


**Section 2** [Adding General information](#general)
- [Population](#pop)
- [Gross Domestic Product](#gdp)
- [Political Stability](#stab)
- [European Union countries](#eu)
    
    
**Section 3** [Adding Food-related data](#food)
- [Food-Balance](#fb)
- [Diet - Calories and Proteins](#diet)
- [Poulty key data](#poultry)
- [Chicken importations](#chicken)

**Section 4** [Imputating missing data](#imputation)
    
**Section 5** [Exporting Data Country](#export) 

<hr>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

<h2>Importing FAOSTAT Datasets</h2><a id='import'></a>

Data are collected from The Food and Agriculture Organization Corporate Statistical Database (FAOSTAT). The FAOSTAT website disseminates statistical data collected and maintained by the Food and Agriculture Organization (FAO).

FAOSTAT - Data collected (2013 and 2017 are selected for the below analysis): analysis is based on 2017 figures, with a comparaison from 2013 data. 
2017 is the most recent comprehensive data from FAOSTAT.

- Datasets Food Balance Animal and Vegetal
- Dataset Population
- Dataset Gross Domestic Product (GDP)
- Dataset Political Stability Index
- Dataset Trade Mix - Chicken 

In [2]:
#import food datasets
veg2013 = pd.read_csv("data/raw/vegetal2013.csv")
ani2013 = pd.read_csv("data/raw/animal2013.csv")
veg2017 = pd.read_csv('data/raw/vegetal2017.csv', dtype={'Note': str })
ani2017 = pd.read_csv("data/raw/animal2017.csv")
ani2013["origin"] = "animal"
veg2013["origin"] = "vegetal"
ani2017["origin"] = "animal"
veg2017["origin"] = "vegetal"

#import population
pop = pd.read_csv("data/raw/FAOSTAT_data_Population_2013-2017.csv")

# import GDP and Stability
gdp = pd.read_csv("data/raw/FAOSTAT_data_MacroIndicators_20132017.csv")
stability = pd.read_csv("data/raw/FAOSTAT_data_PoliticalStability_2017.csv")

# import chicken trade
chicken_trade = pd.read_csv("data/raw/FAOSTAT_data_Trade_Chicken_20132017.csv")

# import EU countries
eu_country = pd.read_csv("data/raw/listofeucountries_csv")

<hr>

<h2>Adding general information</h2><a id='general'></a>

<h3>Population</h3><a id='pop'></a>

Variables added to the dataset:
- Population per country
- Growth of the population (%) - 2017 vs 2013

In [3]:
# population
pop.columns = ["xx","xx2","country_code","country", 'xx3', 'xx4', 'xx5','xx6', 
               'xx7', 'year',"xx8","population_total_thousand","xx9", 'xx10', 'xx11']

pop = pop.drop(["xx","xx2",'xx3','xx4','xx5', 'xx6', 'xx7','xx8', 'xx9', 'xx10', 'xx11'], axis = 1)
pop.reset_index(drop=True, inplace=True)

data_country = pop.pivot_table(index=['country_code', 'country'], columns = ['year'], values=['population_total_thousand'])
data_country.columns = ['pop_2013_thousand', 'pop_thousand']
data_country.reset_index(inplace=True)

#growth population
data_country['pop_growth'] = (data_country['pop_thousand'] / data_country['pop_2013_thousand'] - 1) *100

<h3>Gross Domestic Product (GDP) per capita</h3><a id='gdp'></a>

- GDP per capita (USD)
- GDP Growth - 2017 vs 2013

In [4]:
gdp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "year", "xx7", "value", 'xx8', 'xx9', 'xx10']
gdp = gdp.loc[gdp['item'] == 'Gross Domestic Product per capita'].pivot_table(\
                                                                                                      index=['country_code', "country"], columns = ['year', 'item'], values=['value'])
gdp.columns = ['gdp_percapita_usd_2013', 'gdp_percapita_usd']
gdp.reset_index(inplace=True)

data_country = pd.merge(data_country, gdp, how='left')

GDP of Taiwan is missing. 
Imputation with information collected from the International Monetary Fund (IMF).
- Taiwan GDP per capita 2013 : 43 831 USD 
- Taiwan GDP per capita 2017 : 50 593 USD 
Source: https://bit.ly/2U2251b

In [5]:
# Imputation GDP Taiwan
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2013'] = 43831
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd'] = 50593

In [6]:
# Growth GDP
data_country['gdp_growth'] = np.where(data_country['gdp_percapita_usd_2013'] != 0,
                                                             (data_country['gdp_percapita_usd'] / data_country['gdp_percapita_usd_2013'] - 1) * 100, 0)

In [7]:
#remove column not needed for the analysis
data_country.drop(['gdp_percapita_usd_2013'], axis = 1, inplace=True)

<h3>Political Stability</h3><a id='stab'></a>

In [8]:
stability.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "xx8", "xx8", "value", 'xx9', 'xx10', 'xx11']
stability = stability.pivot_table(index=['country_code', 'country'], columns = ['item'], values=['value'])
stability.columns = ['political_stability_index']
stability.reset_index(inplace=True)

data_country = pd.merge(data_country, stability, how='left')

<h3>European Union countries</h3><a id='eu'></a>

In [9]:
#countries with different spellings between eu_country and data_country
eu_country.loc[~eu_country['x'].isin(data_country['country'])]

,x
5,Czech Republic
23,Slovak Republic
27,United Kingdom


In [10]:
# correct names
eu_country.loc[eu_country['x'] == 'Czech Republic', 'x'] = 'Czechia'
eu_country.loc[eu_country['x'] == 'Slovak Republic', 'x'] = 'Slovakia'
eu_country.loc[eu_country['x'] == 'United Kingdom', 'x'] = 'United Kingdom of Great Britain and Northern Ireland'

In [11]:
# flag EU countries
data_country.loc[data_country['country'].isin(eu_country['x']), 'euro_union'] = 'EU'
data_country.loc[~data_country['country'].isin(eu_country['x']), 'euro_union'] = 'Outside EU'

<hr>

<h2>Adding Food-related data</h2><a id='food'></a>

<h3>Food Balance</h3><a id='fb'></a>

In [12]:
# group food datatsets
temp = [veg2013, ani2013, ani2017, veg2017]
temp = pd.concat(temp, ignore_index=True)

# delete ani2013, veg2013, ani2017, veg2017
del ani2013, veg2013, ani2017, veg2017

temp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'element', 'item_code', 'item',
                'xx4', "year", "unit", "value", 'xx5', 'flag_description', 'origin', 'xx6']

data = temp.pivot_table(index=["year", "country_code", "country", "origin", "item_code", \
                               "item"], columns = ["element"], values=["value"], aggfunc=sum)

# rename columns
data.columns = ['domestic_supply_quantity','export_quantity','fat_supply_quantity_gcapitaday','feed',
                'food','food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr','import_quantity','losses','other_uses','processing',
                'production', 'protein_supply_quantity_gcapitaday', 'residuals', 'seed','stock_variation','tourist_consumption']

data = data.reset_index()

# merge data and pop
data = pd.merge(data, pop, how='left')

Data for China are duplicated. 
<br>data for 'China' = sum of the splitted data for 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan'.
- deletion of 'China' data (code 351)
- 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan' data are kept in order to preserve a better granularity.


Data for Bermuda and Brunei are no longer available in 2017.

In [13]:
# remove doublon from China (code_country 351)
data = data.loc[data.country_code != 351]

# remove Bermuda and Brunei - no information for 2017
data = data.loc[data.country != 'Bermuda']
data = data.loc[data.country != 'Brunei Darussalam']

<h3>Diet - Calories and Proteins</h3><a id='diet'></a>

- Food supply kcal per capita per day - per country
- Protein supply g per capita per day - per country

In [14]:
temp = data.pivot_table(index=['country_code', 'country'], columns=['year'],
                        values=['food_supply_kcalcapitaday',  'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['food_supply_kcalcapitaday_2013', 'protein_supply_gcapitaday_2013', 'food_supply_kcalcapitaday', 'protein_supply_gcapitaday']
temp = temp.reset_index()

data_country = pd.merge(temp, data_country, how='left')

**Animal proteins in the total protein supply**

- Prevalence of the animal proteins in the total proteins - per country
- Growth of the protein supply per country (%) from animals - 2017 vs 2013

In [15]:
NB_DAYS_YEAR = 365 

#total protein supply
data_country['protein_supply_kg'] = \
    data_country['protein_supply_gcapitaday']/1000 * NB_DAYS_YEAR *data_country['pop_thousand']*1000

data_country['protein_supply_kg_2013'] = \
    data_country['protein_supply_gcapitaday_2013']/1000 * NB_DAYS_YEAR *data_country['pop_2013_thousand']*1000

In [16]:
# protein supply from animals
temp = data.loc[data['origin']=='animal'].pivot_table(index=['country_code', 'country'], columns=['year'],
                                                      values=[ 'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['protein_supply_animal_gcapitaday_2013', 'protein_supply_animal_gcapitaday']
temp.reset_index(inplace=True)

data_country = pd.merge(data_country, temp, how='left')

data_country['protein_supply_animal_kg_2013'] = \
    data_country['protein_supply_animal_gcapitaday_2013']/1000 * NB_DAYS_YEAR *data_country['pop_2013_thousand']*1000

data_country['protein_supply_animal_kg'] = \
    data_country['protein_supply_animal_gcapitaday']/1000 * NB_DAYS_YEAR *data_country['pop_thousand']*1000

In [17]:
# prevalence animal proteins in the total protein supply
data_country['prevalence_protein_animal'] = \
    data_country['protein_supply_animal_kg'] / data_country['protein_supply_kg'] * 100

data_country['prevalence_protein_animal_2013'] = \
    data_country['protein_supply_animal_kg_2013'] / data_country['protein_supply_kg_2013'] * 100

In [18]:
# growth protein 
data_country['growth_prev_protein_animal'] = (data_country['prevalence_protein_animal'] / 
                                                      data_country['prevalence_protein_animal_2013'] -1) * 100

In [19]:
#remove column not needed for the analysis
data_country.drop(['protein_supply_animal_gcapitaday_2013', 'protein_supply_animal_kg_2013', 'protein_supply_kg_2013', 
                   'protein_supply_animal_kg', 'protein_supply_kg', 'protein_supply_gcapitaday_2013', 'food_supply_kcalcapitaday_2013', 
                  'prevalence_protein_animal_2013', 'protein_supply_animal_gcapitaday'],
                  axis = 1, inplace=True)

<h3>Poultry (with eggs) key data</h3><a id='poultry'></a>



Poultry are domesticated avian species that can be raised for eggs, meat and/or feathers. The term “poultry” covers a wide range of birds, from indigenous and commercial breeds of chickens to Muscovy ducks, mallard ducks, turkeys, guinea fowl, geese, quail, pigeons, ostriches and pheasants. 

- In 2017, chickens accounted for some 92 percent of the world’s poultry population, followed by ducks (5 percent), and turkeys (2 percent). 
- Chickens contribute 89 percent of world poultry meat production, followed by turkeys with 5 percent, ducks with 4 percent and geese and guinea fowl with 2 percent. The rest comes from other poultry species.
- Chickens provide 92 percent of world egg production.

source: http://www.fao.org/poultry-production-products/production/en/

There is no granular data about chicken exclusively in the dataset 'animal' but the data for the category 'Poultry' (in which chicken is accounting around 90% of the total) give us a fair indicator of the chicken market in each country. 

Poultry composition: 
- Meat chicken, Fat liver prepared (foie gras), Meat chicken canned, Meat,duck, Meat goose and guinea fowl, Meat turkey.
- Eggs are included

In [20]:
temp = data.loc[(data['item'] == 'Poultry Meat')|(data['item'] == 'Eggs')].pivot_table(index=['country_code', 'country'], columns=['year'], \
                                                            values=['domestic_supply_quantity','production', 'import_quantity'], aggfunc=sum)

temp.columns = ['domestic_supply_poultry_tons_2013', 'domestic_supply_poultry_tons', \
                'production_poultry_tons_2013', 'production_poultry_tons', 'import_poultry_tons_2013', 'import_poultry_tons']

temp.reset_index(inplace=True)

#convert 1000 tons to tons
for z in temp.iloc[:, [2,3,4,5,6,7]].columns:
    temp[str(z)] = temp[str(z)] * 1000

data_country = pd.merge(data_country, temp, how='left')

- % growth Poulty Imports (2017 vs 2013)


In [21]:
#growth Poultry imports
data_country['growth_import_poultry'] = \
    np.where(data_country['import_poultry_tons_2013'] != 0, \
             (data_country['import_poultry_tons'] / data_country['import_poultry_tons_2013'] - 1) * 100, 0)

In [22]:
#growth Poultry Domestic Supply
data_country['growth_supply_poultry'] = \
    np.where(data_country['domestic_supply_poultry_tons'] != 0, \
             (data_country['domestic_supply_poultry_tons'] / data_country['domestic_supply_poultry_tons_2013'] - 1) * 100, 0)

In [23]:
#Prevalence of imports in the domestic supply
data_country['prevalence_imports_poultry_supply'] = np.where(data_country['domestic_supply_poultry_tons'] != 0, \
                                                             data_country['import_poultry_tons'] / data_country['domestic_supply_poultry_tons'] * 100, 0)

In [24]:
#Prevalence of Production in the domestic supply
data_country['prevalence_prod_poultry_supply'] = np.where(data_country['domestic_supply_poultry_tons'] != 0, \
                                                             data_country['production_poultry_tons'] / data_country['domestic_supply_poultry_tons'] * 100, 0)

In [25]:
#remove column not needed for the analysis
data_country.drop(['production_poultry_tons_2013', 'import_poultry_tons_2013', 'domestic_supply_poultry_tons_2013'], axis = 1, inplace=True)

<h3>Chicken Importations (with eggs)</h3><a id='chicken'></a>

Data are from the food and agricultural trade dataset collected, processed and disseminated by FAO. The data is mainly provided by UNSD, Eurostat, and other national authorities as needed.
- Import quantity of 'chicken' per country
- Import value of 'chicken' per country

The category 'chicken' is made of 'chicken', 'meat, chicken', 'meat, chicken, canned', 'offals, liver chicken' + 'Eggs' were also included.
<br>Offals liver chicken was not available in the 'Poultry' section on the previous section.  

In [26]:
chicken_trade.columns =['xx', 'xx2', 'country_code', 'country', 'partner_country_code', 'partner_country', 'xx6', \
                        'element', 'xx8', 'xx9', 'xx10', 'year', 'unit', 'value', 'xx13', 'xx14', 'xx15']

# there is a disruptancy on the name of the UK between the FAO trade dataset and other FAO datasets
chicken_trade.loc[chicken_trade['country'] == 'United Kingdom', 'country'] = 'United Kingdom of Great Britain and Northern Ireland'
chicken_trade.loc[chicken_trade['partner_country'] == 'United Kingdom', 'partner_country'] = 'United Kingdom of Great Britain and Northern Ireland'

In [27]:
#Imports of chicken per country - in quantity and value

chicken_import = chicken_trade.loc[((chicken_trade['element'] == 'Import Value') | (chicken_trade['element'] == 'Import Quantity')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

chicken_import = chicken_import.pivot_table(index=["country_code", "country"], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_import.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', 'import_chicken_quantity_tons', \
                         'import_chicken_value_thousand_usd']
chicken_import.reset_index(inplace=True)

data_country = pd.merge(data_country, chicken_import, how='left')

In [28]:
country_no_report_import_chicken = \
    data_country.loc[(data_country['import_chicken_quantity_tons'].isna()) | (data_country['import_chicken_quantity_tons_2013'].isna())]

{{ len(country_no_report_import_chicken)}} countries do have missing information about chicken importations due to the absence of reports in 2013 or 2017:

{{country_no_report_import_chicken.country.tolist()}}

We will get an approximation of these imports through the declaration of exports from partner countries.

In [29]:
# getting information about the chicken imports on the countries not reporting the imports through the export reports from other countries.

chicken_export = chicken_trade.loc[((chicken_trade['element'] == 'Export Quantity') | (chicken_trade['element'] == 'Export Value')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

# Export of chicken to countries not reporting any chicken imports
chicken_export_to_countries_noreport = chicken_export.loc[chicken_export['partner_country'].isin(country_no_report_import_chicken.country)]

chicken_export_to_countries_noreport = chicken_export_to_countries_noreport.pivot_table(\
                                                                                       index=['partner_country_code', 'partner_country'], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_export_to_countries_noreport.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', \
                                                'import_chicken_quantity_tons', 'import_chicken_value_thousand_usd']
chicken_export_to_countries_noreport.reset_index(inplace=True)

chicken_export_to_countries_noreport.rename(columns = {'partner_country_code':'country_code', 'partner_country':'country'}, inplace=True)

In [30]:
data_country.set_index('country', inplace=True)
chicken_export_to_countries_noreport.set_index('country', inplace=True)

#overlapping values from chicken export to data_country for the countries not reporting their imports
data_country = data_country.combine_first(chicken_export_to_countries_noreport)
data_country.reset_index(inplace=True)

In [31]:
data_country['growth_import_chicken_qty'] = \
    (data_country['import_chicken_quantity_tons'] / data_country['import_chicken_quantity_tons_2013'] - 1) *100

data_country['price_import_chicken_usd_tons'] = \
    data_country['import_chicken_value_thousand_usd'] * 1000 / data_country['import_chicken_quantity_tons']

In [32]:
#remove column not needed for the analysis
data_country.drop(['import_chicken_value_thousand_usd_2013', 'import_chicken_quantity_tons_2013',\
                  'import_chicken_value_thousand_usd'], axis = 1, inplace=True)

<hr>

<h2>Imputing missing data</h2><a id='imputation'></a>

Countries with no political stability index: {{data_country.loc[data_country['political_stability_index'].isna()].country.tolist()}}

- Imputation possible for the 2 French territories (French Polynesia and New Caledonia) with France's index.
- Imputation possible for Namibia with United Arab Emirates' index - the political stability index of the UAE is the closest (and almost equivalent) of Namibia in 2017 (source: World Bank - https://www.theglobaleconomy.com/rankings/wb_political_stability/)

In [33]:
# imputation French Polynesia and New Caledonia
for z in ['French Polynesia', 'New Caledonia']:
    data_country.loc[data_country['country'] == z, 'political_stability_index'] = \
                                            data_country.loc[data_country['country'] == 'France']['political_stability_index'].values

# imputation Namibia
data_country.loc[data_country['country'] == 'Namibia', 'political_stability_index'] = \
                                            data_country.loc[data_country['country'] == 'United Arab Emirates']['political_stability_index'].values

<hr>

<h2>Exporting Dataset data_country</h2><a id='export'></a>

In [34]:
#remove column not needed for the analysis
data_country.drop(['pop_2013_thousand', 'country_code'], axis = 1, inplace=True)

In [35]:
data_country.isna().any()

country                              False
domestic_supply_poultry_tons         False
euro_union                           False
food_supply_kcalcapitaday            False
gdp_growth                           False
gdp_percapita_usd                    False
growth_import_poultry                False
growth_prev_protein_animal           False
growth_supply_poultry                False
import_chicken_quantity_tons         False
import_poultry_tons                  False
political_stability_index            False
pop_growth                           False
pop_thousand                         False
prevalence_imports_poultry_supply    False
prevalence_prod_poultry_supply       False
prevalence_protein_animal            False
production_poultry_tons              False
protein_supply_gcapitaday            False
growth_import_chicken_qty            False
price_import_chicken_usd_tons        False
dtype: bool

<h3>Export simplified dataset</h3>

In [36]:
data_country_simplified = data_country[['country', 'food_supply_kcalcapitaday', 'pop_growth', 'protein_supply_gcapitaday',
                                        'prevalence_protein_animal', 'pop_thousand', 'euro_union']]

In [37]:
data_country_simplified.to_csv(r'data/output/data_country_simplified.csv', index = False)

In [38]:
data_country_simplified

,country,food_supply_kcalcapitaday,pop_growth,protein_supply_gcapitaday,prevalence_protein_animal,pop_thousand,euro_union
0,Afghanistan,58.26,12.477767,54.09,19.523017,36296.113,Outside EU
1,Albania,111.37,-0.675703,119.50,55.497908,2884.169,Outside EU
2,Algeria,91.92,8.518733,92.85,27.679052,41389.189,Outside EU
3,Angola,57.27,14.610305,54.09,30.449251,29816.766,Outside EU
4,Antigua and Barbuda,83.49,4.272477,81.15,66.605052,95.426,Outside EU
...,...,...,...,...,...,...,...
167,Venezuela (Bolivarian Republic of),72.96,-1.271131,56.30,45.115453,29402.484,Outside EU
168,Viet Nam,81.61,4.240161,90.03,40.875264,94600.648,Outside EU
169,Yemen,58.90,10.687948,53.24,20.060105,27834.819,Outside EU
170,Zambia,55.22,12.910256,59.06,19.776498,16853.599,Outside EU


<h3>Export complete dataset</h3>

data_country : complete dataset
- data_pca : data_country with only the variable required fo the PCA
    - data_pca_poultry : sub dataset of data_pca with only variables related to Poultry 
    - data_pca_not_poultry : sub dataset of data_pca with variables not related to Poultry


In [39]:
data_country

,country,domestic_supply_poultry_tons,euro_union,food_supply_kcalcapitaday,gdp_growth,gdp_percapita_usd,growth_import_poultry,growth_prev_protein_animal,growth_supply_poultry,import_chicken_quantity_tons,...,political_stability_index,pop_growth,pop_thousand,prevalence_imports_poultry_supply,prevalence_prod_poultry_supply,prevalence_protein_animal,production_poultry_tons,protein_supply_gcapitaday,growth_import_chicken_qty,price_import_chicken_usd_tons
0,Afghanistan,131000.0,Outside EU,58.26,-5.206857,468.297893,9.090909,3093.076604,7.377049,70001.0,...,-2.78,12.477767,36296.113,36.641221,63.358779,19.523017,83000.0,54.09,64.086637,1234.310938
1,Albania,97000.0,Outside EU,111.37,7.818559,3347.701760,-1.538462,3076.113231,8.988764,40034.0,...,0.40,-0.675703,2884.169,65.979381,39.175258,55.497908,38000.0,119.50,66.669442,1314.332817
2,Algeria,667000.0,Outside EU,91.92,-7.348644,3264.338962,4.724409,3606.422327,4.381847,1991.0,...,-0.96,8.518733,41389.189,99.700150,0.299850,27.679052,2000.0,92.85,-26.774549,12734.304370
3,Angola,347000.0,Outside EU,57.27,-15.509934,2805.692595,34.285714,3649.891485,-13.466334,267388.0,...,-0.29,14.610305,29816.766,13.544669,86.455331,30.449251,300000.0,54.09,-22.429454,1014.503269
4,Antigua and Barbuda,7000.0,Outside EU,83.49,15.033876,10549.721325,0.000000,2746.800496,0.000000,6455.0,...,0.76,4.272477,95.426,0.000000,100.000000,66.605052,7000.0,81.15,9.965928,1979.705655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Venezuela (Bolivarian Republic of),790000.0,Outside EU,72.96,-38.559352,6042.349852,-32.088889,2558.652576,-43.852168,25033.0,...,-1.18,-1.271131,29402.484,96.708861,3.164557,45.115453,25000.0,56.30,-91.089620,2464.027484
168,Viet Nam,1740000.0,Outside EU,81.61,23.679433,1619.296641,43.422354,3767.681966,15.308151,363681.0,...,0.31,4.240161,94600.648,83.333333,16.954023,40.875264,295000.0,90.03,-30.725185,1082.748343
169,Yemen,312000.0,Outside EU,58.90,-49.953849,515.814155,6.363636,2886.565840,-5.167173,77952.0,...,-2.96,10.687948,27834.819,75.000000,25.000000,20.060105,78000.0,53.24,-28.257989,1554.674672
170,Zambia,111000.0,Outside EU,55.22,-3.626393,1115.460621,0.000000,4202.406169,8.823529,12070.0,...,0.11,12.910256,16853.599,90.990991,10.810811,19.776498,12000.0,59.06,669.279796,921.043911


In [40]:
data_country.to_csv(r'data/output/data_country.csv', index = False)

In [41]:
# variables to be used on the PCA 
data_pca = data_country[['country',
       'food_supply_kcalcapitaday', 'gdp_growth', 'gdp_percapita_usd',
       'growth_prev_protein_animal', 
       'growth_import_poultry', 'growth_supply_poultry',
       'political_stability_index', 'pop_growth',
       'prevalence_imports_poultry_supply', 'prevalence_prod_poultry_supply',
       'prevalence_protein_animal',
        'protein_supply_gcapitaday',
       'growth_import_chicken_qty', 'price_import_chicken_usd_tons']]

In [42]:
data_pca.to_csv(r'data/output/data_pca.csv', index = False)

In [43]:
# subset of data_pca with only data related to Poultry and Chicken 
data_pca_poultry = data_country[['country',
       'growth_import_poultry', 'growth_supply_poultry', 'prevalence_imports_poultry_supply', 'prevalence_prod_poultry_supply',
       'growth_import_chicken_qty', 'price_import_chicken_usd_tons']]

In [44]:
data_pca_poultry.to_csv(r'data/output/data_pca_poultry.csv', index = False)

In [45]:
# subset of data_pca with data NOT related to Poultry and Chicken 
data_pca_not_poultry = data_pca.copy()
for i in data_pca.columns: 
    if i in data_pca_poultry.columns and i != 'country':
        data_pca_not_poultry.drop([i], axis = 1, inplace=True)


In [46]:
data_pca_not_poultry.to_csv(r'data/output/data_pca_not_poultry.csv', index = False)